In [16]:
#load library
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
import numpy as np

In [17]:
#set global Variables/hyperparameters
batchSize = 100
orginalDim =28*28
latentDim = 2
intermediateDim = 256
epochs = 50
epsilonSTD = 1.0

In [18]:
#Sampling function 
def sampling(args: tuple):
    zMean, zLogVar = args
    epsilon = K.random_normal(shape=(K.shape(zMean)[0],latentDim),mean=0.,stddev=epsilonSTD)
    return zMean+K.exp(zLogVar/2)*epsilon

In [19]:
#Creating encoder
x = Input(shape=(orginalDim),name="input")
h = Dense(intermediateDim,activation="relu", name="encoding")(x)
#mean of the latent space
zMean= Dense(latentDim,name="mean")(h)
#log variance of the latent space
zLogVar = Dense(latentDim, name="log-variance")(h)

z= Lambda(sampling, output_shape=(latentDim,))((zMean,zLogVar))
encoder = Model(x,[zMean,zLogVar,z], name = "encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 784)]        0           []                               
                                                                                                  
 encoding (Dense)               (None, 256)          200960      ['input[0][0]']                  
                                                                                                  
 mean (Dense)                   (None, 2)            514         ['encoding[0][0]']               
                                                                                                  
 log-variance (Dense)           (None, 2)            514         ['encoding[0][0]']               
                                                                                            